# pip & import

In [ ]:
!pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 1.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import category_encoders as ce

In [ ]:
apply_train_df = pd.read_csv('apply_train.csv')
resume_train_df = pd.read_csv('resume.csv')
certificate_df = pd.read_csv('resume_certificate.csv')
education_train_df = pd.read_csv('resume_education.csv')
language_train_df = pd.read_csv('resume_language.csv')
company_train_df = pd.read_csv('company.csv')
recruitment_train_df = pd.read_csv('recruitment.csv')

# 함수 정의 & 글로벌 변수

In [ ]:
def recall5(answer_df, submission_df):

    primary_col = answer_df.columns[0]
    secondary_col = answer_df.columns[1]

    # submission의 예측이 각각 5개인지 확인
    prediction_counts = submission_df.groupby(primary_col).size()
    if not all(prediction_counts == 5):
        raise ValueError(f"Each {primary_col} should have exactly 5 {secondary_col} predictions.")


    # submission의 예측된 값들에 null값이 있는지 확인
    if submission_df[secondary_col].isnull().any():
        raise ValueError(f"Predicted {secondary_col} contains NULL values.")

    # 예측값에 중복이 있는지 확인
    duplicated_preds = submission_df.groupby(primary_col).apply(lambda x: x[secondary_col].duplicated().any())
    if duplicated_preds.any():
        raise ValueError(f"Predicted {secondary_col} contains duplicates for some {primary_col}.")


    # Filter the submission dataframe based on the primary_col present in the answer dataframe
    # primary_col 즉 resume_seq가 양측에 있는지 확인 후 남김
    submission_df = submission_df[submission_df[primary_col].isin(answer_df[primary_col])]

    # For each primary_col, get the top 5 predicted secondary_col values
    #
    top_5_preds = submission_df.groupby(primary_col).apply(lambda x: x[secondary_col].head(5).tolist()).to_dict()

    # Convert the answer_df to a dictionary for easier lookup
    true_dict = answer_df.groupby(primary_col).apply(lambda x: x[secondary_col].tolist()).to_dict()


    individual_recalls = []
    for key, val in true_dict.items():
        if key in top_5_preds:
            correct_matches = len(set(true_dict[key]) & set(top_5_preds[key]))
            individual_recall = correct_matches / min(len(val), 5) # 공정한 평가를 가능하게 위하여 분모(k)를 'min(len(val), 5)' 로 설정함
            individual_recalls.append(individual_recall)


    recall = np.mean(individual_recalls)
    return recall

In [ ]:
def minimum_condition(resume, company):
  if(resume['degree'] < company['education']):
    return False
  else:
    return True
   # if(resume['career_month']>company['career_end'] or resume['career_month']<company['career_start']): 다 0임
     # return False
   # else:


In [ ]:
a = 1
b = 0
c = 0

In [ ]:
# user = f'U{str(user+1).zfill(5)}'

# apply_matrix

In [ ]:
#학습, 검증 분리
train, test = [], []
apply_train_groupby = apply_train_df.groupby('resume_seq')['recruitment_seq'].apply(list)
for uid, iids in zip(apply_train_groupby.index.tolist(), apply_train_groupby.values.tolist()):
    for iid in iids[:-1]:
        train.append([uid,iid])
    test.append([uid, iids[-1]])

In [ ]:
apply_train = pd.DataFrame(train, columns=['resume_seq', 'recruitment_seq'])
apply_test = pd.DataFrame(test, columns=['resume_seq', 'recruitment_seq'])
apply_pred = apply_train_df.copy()

In [ ]:
# 사용자-아이템 행렬 생성: 구직자가 해당 채용 공고에 지원했으면 1, 아니면 0으로 설정
apply_train_item_matrix = apply_train.groupby(['resume_seq', 'recruitment_seq']).size().unstack(fill_value=0)
apply_train_similarity = cosine_similarity(apply_train_item_matrix) # 사용자 간의 유사성 계산

apply_test_item_matrix = apply_test.groupby(['resume_seq', 'recruitment_seq']).size().unstack(fill_value=0)
apply_test_similarity = cosine_similarity(apply_test_item_matrix) # 사용자 간의 유사성 계산

apply_pred_item_matrix = apply_pred.groupby(['resume_seq', 'recruitment_seq']).size().unstack(fill_value=0)
apply_pred_similarity = cosine_similarity(apply_pred_item_matrix) # 사용자 간의 유사성 계산

In [ ]:
apply_train_matrix = pd.DataFrame(apply_train_similarity) # DataFrame 생성
apply_test_matrix = pd.DataFrame(apply_test_similarity) # DataFrame 생성
apply_pred_matrix = pd.DataFrame(apply_pred_similarity) # DataFrame 생성

# 행과 열 이름 설정
apply_train_matrix.columns = [f'U{i:05d}' for i in range(1, 8483)]
apply_train_matrix.index = [f'U{i:05d}' for i in range(1, 8483)]

apply_test_matrix.columns = [f'U{i:05d}' for i in range(1, 8483)]
apply_test_matrix.index = [f'U{i:05d}' for i in range(1, 8483)]

apply_pred_matrix.columns = [f'U{i:05d}' for i in range(1, 8483)]
apply_pred_matrix.index = [f'U{i:05d}' for i in range(1, 8483)]

# resume_matrix

In [ ]:
resume_train = resume_train_df.copy()
resume_train = resume_train[['resume_seq', 'degree', 'job_code_seq1']] # 여기 수정
resume_train['degree'] = resume_train['degree'].astype("category") # 원핫 인코딩 할 것들(숫자)

In [ ]:
_train_encoded = pd.DataFrame()

for col in ['degree']: # 수정
  _encoder = ce.OneHotEncoder(use_cat_names=True)
  _encoder.fit(resume_train[col])
  _encoded = _encoder.transform(resume_train[col])
  _train_encoded = pd.concat([_train_encoded, _encoded], axis=1)

In [ ]:
resume_tmp = resume_train[['resume_seq']]
resume_tmp = pd.merge(resume_tmp, _train_encoded, left_index=True, right_index=True, how='inner')

In [ ]:
resume_item_matrix = resume_train.groupby(['resume_seq', 'job_code_seq1']).size().unstack(fill_value=0)
resume_item_matrix = pd.merge(resume_tmp, resume_item_matrix, on='resume_seq', how='left')
resume_item_matrix.set_index('resume_seq', inplace=True)
resume_similarity = cosine_similarity(resume_item_matrix) # 사용자 간의 유사성 계산

In [ ]:
resume_matrix = pd.DataFrame(resume_similarity) # DataFrame 생성
# 행과 열 이름 설정
resume_matrix.columns = [f'U{i:05d}' for i in range(1, 8483)]
resume_matrix.index = [f'U{i:05d}' for i in range(1, 8483)]

# certificate

In [ ]:
certificate_train = certificate_df.copy()
tmp = apply_train_df.copy()
tmp = tmp.groupby('resume_seq')['recruitment_seq'].apply(';'.join).reset_index()
tmp = tmp.drop(columns = ['recruitment_seq'])

In [ ]:
certificate_item_matrix = certificate_train.groupby(['resume_seq', 'certificate_contents']).size().unstack(fill_value=0)
certificate_item_matrix = pd.merge(tmp, certificate_item_matrix, on='resume_seq', how='left')
certificate_item_matrix.fillna(0, inplace=True)
certificate_item_matrix.set_index('resume_seq', inplace=True)
certificate_similarity = cosine_similarity(certificate_item_matrix) # 사용자 간의 유사성 계산

In [ ]:
certificate_matrix = pd.DataFrame(certificate_similarity) # DataFrame 생성
# 행과 열 이름 설정
certificate_matrix.columns = [f'U{i:05d}' for i in range(1, 8483)]
certificate_matrix.index = [f'U{i:05d}' for i in range(1, 8483)]

# education

In [ ]:
education_train = education_train_df.copy()
education_train

,resume_seq,hischool_type_seq,hischool_special_type,hischool_nation,hischool_gender,hischool_location_seq,univ_type_seq1,univ_type_seq2,univ_transfer,univ_location,univ_major,univ_sub_major,univ_major_type,univ_score
0,U01419,21,일반고,사립,남자학교,3,5,5,0,3,NaN,NaN,9,60.0
1,U03375,21,일반고,사립,여자학교,3,5,5,0,3,NaN,NaN,4,80.0
2,U06523,21,일반고,사립,남여공학,3,5,5,0,3,NaN,NaN,8,70.0
3,U06619,21,일반고,사립,남여공학,5,5,5,0,5,NaN,NaN,8,80.0
4,U05015,16,특성화고,공립,남여공학,3,5,5,0,3,NaN,NaN,9,80.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8477,U06326,21,일반고,사립,여자학교,9,5,5,0,7,섬우패션학부의류패션전공,NaN,10,70.0
8478,U07284,0,기타,기타,기타,0,5,5,0,11,NaN,NaN,3,70.0
8479,U04109,21,일반고,공립,남여공학,5,0,0,0,0,NaN,NaN,9,80.0
8480,U01863,21,일반고,사립,여자학교,15,3,5,0,15,의류학과생활과학계열,NaN,9,80.0


# language

# 회사 최소 조건

In [ ]:
resume_c = resume_train_df.copy()
recruitment_c = recruitment_train_df.copy()

In [ ]:
# 'resume_seq' 열을 기준으로 정렬
resume_c = resume_c.sort_values(by='resume_seq')
# 인덱스를 순차적으로 변경
resume_c['resume_seq'] = [f'U{i:05}' for i in range(1, len(resume_c) + 1)]
resume_c = resume_c.set_index('resume_seq')

# 'resume_seq' 열을 기준으로 정렬
recruitment_c = recruitment_c.sort_values(by='recruitment_seq')
# 인덱스를 순차적으로 변경
recruitment_c['recruitment_seq'] = [f'R{i:05}' for i in range(1, len(recruitment_c) + 1)]
recruitment_c = recruitment_c.set_index('recruitment_seq')

In [ ]:
resume_c = resume_c[['degree']]
recruitment_c = recruitment_c[['education']]

In [ ]:
apply_item_matrix_new = []
count = 0
# 초등학생넘들
tem_6 = [] # U05886이 6학년
tem_5 = [] # U08422이 5학년
tem_4 = [] # U04600이 4학년
tem_3 = [] # U03400이 3학년
tem_2 = [] # U04220이 2학년

for recruitment_seq in range(len(recruitment_c)): #
  if resume_c.loc['U05886']['degree'] < recruitment_c.iloc[recruitment_seq]['education']:
        a = False
  else:
        a = True
  tem_6.append(a)

for recruitment_seq in range(len(recruitment_c)):
  if resume_c.loc['U08422']['degree'] < recruitment_c.iloc[recruitment_seq]['education']:
        a = False
  else:
        a = True
  tem_5.append(a)

for recruitment_seq in range(len(recruitment_c)):
  if resume_c.loc['U04600']['degree'] < recruitment_c.iloc[recruitment_seq]['education']:
        a = False
  else:
        a = True
  tem_4.append(a)

for recruitment_seq in range(len(recruitment_c)):
  if resume_c.loc['U03400']['degree'] < recruitment_c.iloc[recruitment_seq]['education']:
        a = False
  else:
        a = True
  tem_3.append(a)

for recruitment_seq in range(len(recruitment_c)):
  if resume_c.loc['U04220']['degree'] < recruitment_c.iloc[recruitment_seq]['education']:
        a = False
  else:
        a = True
  tem_2.append(a)

# apply_item_matrix의 인덱스(=resume_seq)와 컬럼(=recruitment_seq)을 뽑아냄
for resume_seq in range(len(resume_c)):
    if resume_c.iloc[resume_seq]['degree'] == 6:
      apply_item_matrix_new.append(tem_6)
    elif resume_c.iloc[resume_seq]['degree'] == 5:
      apply_item_matrix_new.append(tem_5)
    elif resume_c.iloc[resume_seq]['degree'] == 4:
      apply_item_matrix_new.append(tem_4)
    elif resume_c.iloc[resume_seq]['degree'] == 3:
      apply_item_matrix_new.append(tem_3)
    else:
      apply_item_matrix_new.append(tem_2)

# 새로운 DataFrame 생성
apply_item_matrix_new = pd.DataFrame(apply_item_matrix_new, index=apply_pred_item_matrix.index, columns=apply_pred_item_matrix.columns)
apply_item_matrix_new

recruitment_seq,R00001,R00002,R00003,R00004,R00005,R00006,R00007,R00008,R00009,R00010,...,R06686,R06687,R06688,R06689,R06690,R06691,R06692,R06693,R06694,R06695
resume_seq,,,,,,,,,,,,,,,,,,,,,
U00001,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
U00002,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
U00003,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
U00004,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
U00005,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
U08478,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
U08479,True,True,True,True,True,False,True,True,True,True,...,True,True,True,True,True,True,True,False,False,True
U08480,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


# 결과 도출

In [ ]:
# train_사용자 간의 유사성 계산
user_train_similarity = a*apply_train_similarity + b*resume_similarity + c*certificate_similarity
user_predicted_scores = user_train_similarity.dot(apply_train_item_matrix) / np.array([np.abs(user_train_similarity).sum(axis=1)]).T

<ipython-input-25-98e1e26de64b>:3: RuntimeWarning: invalid value encountered in divide
  user_predicted_scores = user_train_similarity.dot(apply_train_item_matrix) / np.array([np.abs(user_train_similarity).sum(axis=1)]).T


In [ ]:
user_matrix = pd.DataFrame(user_train_similarity) # DataFrame 생성
# 행과 열 이름 설정
user_matrix.  columns = [f'U{i:05d}' for i in range(1, 8483)]
user_matrix.index = [f'U{i:05d}' for i in range(1, 8483)]

In [ ]:
sort_user_list = []
applied_jobs = []
for idx, user in enumerate(user_matrix.index):
    # 해당 사용자가 지원한 채용 공고
    applied_job = apply_train[apply_train['resume_seq'] == user]['recruitment_seq'].values
    applied_jobs.append([user, applied_job])

    # 해당 사용자의 최종 유사도 (높은 유사도부터 정렬)
    sorted_user_indices = user_train_similarity[idx].argsort()[::-1]
    sort_user_list.append([user,sorted_user_indices])

In [ ]:
# 이미 지원한 채용 공고 제외하고 추천
recommendations = []
for idx, user in enumerate(apply_train_item_matrix.index):
    # 해당 사용자가 지원한 채용 공고
    applied_jobs = set(apply_train_item_matrix.loc[user][apply_train_item_matrix.loc[user] == 1].index)

    # 해당 사용자의 추천 점수 (높은 점수부터 정렬)
    sorted_job_indices = user_predicted_scores[idx].argsort()[::-1]
    recommended_jobs = [job for job in apply_train_item_matrix.columns[sorted_job_indices] if (job not in applied_jobs) and (apply_item_matrix_new.loc[user].loc[job])][:5]

    for job in recommended_jobs:
        recommendations.append([user, job])

In [ ]:
val_prediction = pd.DataFrame(recommendations, columns=['resume_seq', 'recruitment_seq'])

In [ ]:
recall5(apply_test,val_prediction)

0.0010610705022400376

# pred 결과 도출

In [ ]:
# 사용자 간의 유사성 계산
user_similarity = a*apply_pred_similarity + b*resume_similarity+ c*certificate_similarity
user_predicted_scores = user_similarity.dot(apply_pred_item_matrix) / np.array([np.abs(user_similarity).sum(axis=1)]).T

<ipython-input-31-0c4eda969dbe>:3: RuntimeWarning: invalid value encountered in divide
  user_predicted_scores = user_similarity.dot(apply_pred_item_matrix) / np.array([np.abs(user_similarity).sum(axis=1)]).T


In [ ]:
user_matrix = pd.DataFrame(user_similarity) # DataFrame 생성
# 행과 열 이름 설정
user_matrix.columns = [f'U{i:05d}' for i in range(1, 8483)]
user_matrix.index = [f'U{i:05d}' for i in range(1, 8483)]

In [ ]:
sort_user_list = []
applied_jobs = []
for idx, user in enumerate(user_matrix.index):
    # 해당 사용자가 지원한 채용 공고
    applied_job = apply_train[apply_train['resume_seq'] == user]['recruitment_seq'].values
    applied_jobs.append([user, applied_job])

    # 해당 사용자의 최종 유사도 (높은 유사도부터 정렬)
    sorted_user_indices = user_similarity[idx].argsort()[::-1]
    sort_user_list.append([user,sorted_user_indices])

In [ ]:
# 이미 지원한 채용 공고 제외하고 추천
recommendations = []
for idx, user in enumerate(apply_pred_item_matrix.index):
    # 해당 사용자가 지원한 채용 공고
    applied_jobs = set(apply_pred_item_matrix.loc[user][apply_pred_item_matrix.loc[user] == 1].index)

    # 해당 사용자의 추천 점수 (높은 점수부터 정렬)
    sorted_job_indices = user_predicted_scores[idx].argsort()[::-1]
    recommended_jobs = [job for job in apply_pred_item_matrix.columns[sorted_job_indices] if (job not in applied_jobs) and (apply_item_matrix_new.loc[user].loc[job])][:5]

    for job in recommended_jobs:
        recommendations.append([user, job])

KeyboardInterrupt: ignored

In [ ]:
# sample_submission.csv 형태로 DataFrame 생성
top_recommendations = pd.DataFrame(recommendations, columns=['resume_seq', 'recruitment_seq'])

top_recommendations.to_csv('./S_submit.csv', index=False)

1. 사람이 사는 곳 회사 있는 곳 비교  
2.